In [1]:
import neuraltda.test_pipeline as ttop
import numpy as np
import neuraltda.topology as top
reload(ttop)
reload(top)

/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


<module 'neuraltda.topology' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/topology.pyc'>

In [4]:
block_path = '/Users/brad/experiments/B999/'
kwikfile = 'B999_S00_P00.kwik'
bin_id = '0801161542'
bin_def_file = '/Users/brad/GentnerLab/code/NeuralTDA/standard_good_binnings.txt'
n_cells = 60
maxt = 6
fs = 24000.0
dthetadt = 2*np.pi/6
kappa = 1
maxfr = 25
ncellperm = 40
nperms = 1


In [5]:
spikes, clusters, trials = ttop.generate_test_dataset(n_cells, maxt, fs, dthetadt, kappa, maxfr)

[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   1.43998000e+05
   1.43999000e+05   1.44000000e+05]


In [6]:
spikes

,cluster,recording,time_samples
488,16.0,0.0,47.0
1627,54.0,0.0,91.0
489,16.0,0.0,122.0
1596,53.0,0.0,134.0
0,0.0,0.0,179.0
798,26.0,0.0,187.0
241,8.0,0.0,396.0
388,13.0,0.0,552.0
23,1.0,0.0,554.0
591,19.0,0.0,685.0


In [ ]:
popvec_fname = '/Users/brad/experiments/B999/testbin.binned'
segment_info = {'period': 1}
cluster_group = ['Good']
win_size = 50.0
top.build_population_embedding(spikes, trials, clusters, win_size, fs, cluster_group, segment_info, popvec_fname)

In [3]:
ttop.test_pipeline(block_path, kwikfile, bin_id, bin_def_file, n_cells, maxt, fs, dthetadt, kappa, maxfr, ncellperm, nperms)

[  0.00000000e+00   1.00000000e+00   2.00000000e+00 ...,   1.43998000e+05
   1.43999000e+05   1.44000000e+05]


/Users/brad/anaconda/envs/neuraltda/lib/python2.7/site-packages/numpy/lib/shape_base.py:873: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return c.reshape(shape_out)
/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/test_pipeline.py:37: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  spikes_dataframe = spikes_dataframe.sort(columns='time_samples')


      cluster  recording  time_samples
2288     39.0        0.0          17.0
1972     33.0        0.0          59.0
359       6.0        0.0          76.0
2617     44.0        0.0          85.0
360       6.0        0.0          93.0
Binning data into good-50ms-stim.binned
Done
Binning data into good-100ms-stim.binned
Done
Binning data into good-150ms-stim.binned
Done
Binning data into good-200ms-stim.binned
Done
Binning data into good-25ms-stim.binned
Done
Binning data into good-50ms-prestim.binned
Done
Binning data into good-100ms-prestim.binned
Done
Binning data into good-150ms-prestim.binned
Done
Binning data into good-200ms-prestim.binned
Done
Binning data into good-25ms-prestim.binned
Done


In [ ]:
import pickle


In [ ]:
pickle.load('/Volumes/gentner/btheilma/experiments/B999/topology/0801161509/B999_S00_P00-good-50ms-stim-permuted/B999_S00_P00-good-50ms-stim-permuted-0801161509-stim-test_pipeline_stimulus-bettiPersistence.pkl')